Quante evidenze archeologiche di epoca romana sono presenti in Italia?

Curiosando sul [Catalogo generale dei Beni Culturali](https://catalogo.beniculturali.it/) mi aspettavo di trovare una risposta ma non è stato così. Già solo accedendo al portale, senza applicare nessun tipo di filtro, effettuando uno zoom sull'area dei Campi Flegrei mi ritrovo nella situazione che segue.

<p align="center">
  <img width="100%" height="800px" src="zoom_campi_flegrei.png">
</p>

E' presente il solo [Castello di Baia](https://www.campiflegreionline.it/Castello_di_Baia.html) in questa area, secondo il catalogo, ma nella realtà in quest'area possiamo trovare:
- la [Piscina Mirabilis](https://piscinamirabilisbacoli.it/it)
- la [Casina Vanvitelliana](https://www.prolococittadibacoli.it/casina-vanvitelliana/)
- [Baia sommersa](https://www.baiasommersa.it/)
- il [Tempio di Serapide](http://www.pafleg.it/it/4388/localita/67/macellum-tempio-di-serapide)
- il [Parco archeologico del Pausilypon](https://www.areamarinaprotettagaiola.it/pausilypon)

Solo per citare i luoghi che conosco per averli visitati direttamente, ma ce ne sono tanti altri.

Così ho deciso di usare i superpoteri di Python e [OpenStreetMap](https://www.openstreetmap.org/) per darmi una risposta.

## Prima di iniziare


### Librerie
Per raggiungere l'obiettivo userò le librerie che seguono.

In [1]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
import osmnx as ox

### Fonti dati
Oltre ai dati di OpenStreetMap userò i confini comunali generalizzati 2023 rilasciati da ISTAT.


In [2]:
main_folder = Path.cwd().parent.parent.parent.parent.joinpath('sample_data')
municipalities = main_folder.joinpath('Com01012023_g').joinpath('Com01012023_g_WGS84.shp')

municipalities_gdf = gpd.read_file(municipalities).to_crs(4326)

# Comuni campani
target_region = municipalities_gdf[municipalities_gdf['COD_REG']==15]

target_region

,COD_RIP,COD_REG,COD_PROV,COD_CM,COD_UTS,PRO_COM,PRO_COM_T,COMUNE,COMUNE_A,CC_UTS,Shape_Leng,geometry
4951,4,15,61,0,61,61001,061001,Ailano,NaN,0,20581.746400,"POLYGON ((14.21374 41.40203, 14.21459 41.40122..."
4952,4,15,61,0,61,61002,061002,Alife,NaN,0,40841.207949,"POLYGON ((14.30840 41.38635, 14.31128 41.38539..."
4953,4,15,61,0,61,61003,061003,Alvignano,NaN,0,28587.631245,"POLYGON ((14.34963 41.28071, 14.35018 41.27727..."
4954,4,15,61,0,61,61004,061004,Arienzo,NaN,0,17505.659927,"POLYGON ((14.51392 41.04895, 14.51823 41.04876..."
4955,4,15,61,0,61,61005,061005,Aversa,NaN,0,16771.036732,"POLYGON ((14.20764 40.98608, 14.21433 40.98588..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5496,4,15,65,0,65,65154,065154,Vallo della Lucania,NaN,0,30456.305184,"POLYGON ((15.23387 40.25431, 15.23454 40.25401..."
5497,4,15,65,0,65,65155,065155,Valva,NaN,0,22401.620108,"POLYGON ((15.26793 40.78811, 15.26828 40.78707..."
5498,4,15,65,0,65,65156,065156,Vibonati,NaN,0,22995.407185,"POLYGON ((15.58320 40.12839, 15.58939 40.12482..."
5499,4,15,65,0,65,65157,065157,Vietri sul Mare,NaN,0,15182.573826,"POLYGON ((14.72267 40.68556, 14.72256 40.68349..."


Come vedremo in seguito tornerà molto utile conoscere la tassonomia di OpenStreetMap, tassonomia disponibile [qui](https://wiki.openstreetmap.org/wiki/Map_features).

## Rimozione dei warnings
Il codice che segue serve solo per eliminare i messaggi di warning che renderebbero antiestetico l'articolo. **Attenzione ad ignorarli però!** Vanno ben approfonditi e non ignorati alla leggera. Io lo faccio solo perchè questo è un articolo dimostrativo.

In [3]:
# import warnings
# warnings.filterwarnings('ignore')
# warnings.simplefilter('ignore')

## Contenuti
- [1. Lavoriamo in piccolo: Bacoli](#1.-Lavoriamo-in-piccolo:-Bacoli)
- [Conclusione](#Conclusione)

# 1. Lavoriamo in piccolo: Bacoli
Partiremo con l'analizzare l'area di Bacoli. 

In [4]:
bacoli = target_region[target_region['COMUNE']=='Bacoli'].squeeze()
bacoli

COD_RIP                                                       4
COD_REG                                                      15
COD_PROV                                                     63
COD_CM                                                      263
COD_UTS                                                     263
PRO_COM                                                   63006
PRO_COM_T                                                063006
COMUNE                                                   Bacoli
COMUNE_A                                                    NaN
CC_UTS                                                        0
Shape_Leng                                         20303.054123
geometry      POLYGON ((14.06724657892957 40.842943679557926...
Name: 5138, dtype: object

Interrogheremo il database di OpenStreetMap usando [OSMnx](https://osmnx.readthedocs.io/en/stable/index.html) libreria che (cito dal sito del progetto) *is a Python package to download, model, analyze, and visualize street networks and other geospatial features from OpenStreetMap*. Andremo in particolare ad usare il TAG **historic** con alcune eccezioni che vedremo.

In [5]:
bacoli_data = ox.features_from_polygon(
    polygon=bacoli.geometry,
    tags={'historic': True}
)

bacoli_data

historic  \
element_type osmid                             
node         1045290525  archaeological_site   
             1104926219  archaeological_site   
             1139094827  archaeological_site   
             1139094838  archaeological_site   
             1706624371  archaeological_site   
             2235447677                ruins   
             2235447711             memorial   
             2235447893             memorial   
             8755033944                ruins   
             9060732167                ruins   
way          124092891   archaeological_site   
             124093354   archaeological_site   
             144917231                castle   
             240100401   archaeological_site   
             323122081              building   
             406222066   archaeological_site   
             429218964   archaeological_site   
             429324871   archaeological_site   
             429338160   archaeological_site   
             429339087   archaeological_site   
             429340403   archaeological_site   
             429341185   archaeological_site   
             429341186   archaeological_site   
             429341188   archaeological_site   
             429341189   archaeological_site   
             429341190   archaeological_site   
             736686146              building   
             736686737   archaeological_site   
             736690821   archaeological_site   
             1154634667  archaeological_site   

                                                     name   wikidata  \
element_type osmid                                                     
node         1045290525                 Piscina Mirabilis    Q222410   
             1104926219   Grotte dell'Acqua - Lago Fusaro        NaN   
             1139094827            Grotta della Dragonara  Q55673920   
             1139094838           Sacello degli Augustali   Q3944423   
             1706624371                 Grotta di Cocceio    Q518104   
             2235447677                               NaN        NaN   
             2235447711                            Bunker        NaN   
             2235447893                            Bunker        NaN   
             8755033944                    Tempo di Iside        NaN   
             9060732167                               NaN        NaN   
way          124092891   Parco Archeologico Terme di Baia   Q3685188   
             124093354                 Anfiteatro di Cuma  Q93976746   
             144917231         Castello Aragonese di Baia   Q3662303   
             240100401                 Tomba di Agrippina        NaN   
             323122081               Casina Vanvitelliana    Q958152   
             406222066                    Tempio di Diana        NaN   
             429218964                          Columbari        NaN   
             429324871            Monumental Park of Baia        NaN   
             429338160                 Tempio di Mercurio        NaN   
             429339087                   Tempio di Venere  Q96694237   
             429340403                                NaN        NaN   
             429341185                                NaN        NaN   
             429341186                                NaN        NaN   
             429341188                                NaN        NaN   
             429341189                                NaN        NaN   
             429341190                                NaN        NaN   
             736686146                                NaN        NaN   
             736686737                                NaN        NaN   
             736690821                                NaN        NaN   
             1154634667          Teatro romano di Misenum  Q55685837   

                                                 wikipedia  \
element_type osmid                                           
node         1045290525               en:Piscina Mirabilis   
        

Come si può vedere in quest'area ci sono ben 35 siti di interesse archeologico, molti più dei tre che conosco che fanno parte dell'elenco di inizio articolo.

Ma veniamo ai TAG di nostro interesse. Analizzando l'elenco delle [chiavi](https://wiki.openstreetmap.org/wiki/Map_features#Historic) ce ne sono alcune che non mi interessano e le inserirò nella viriabile che segue perchè a me interessano solo le strutture o i resti di strutture.

In [6]:
excluded_keys = ['aircraft', 'anchor', 'battlefield', 'bomb_crater', 'boundary_stone', 'cannon', 'cattle_crush', 'district', 'highwater_mark', 'locomotive', 'ogham_stone', 'railway_car', 'road', 'shieling', 'ship', 'tank', 'vehicle', 'wayside_cross', 'wayside_shrine', 'wreck']

excluded_keys

['aircraft',
 'anchor',
 'battlefield',
 'bomb_crater',
 'boundary_stone',
 'cannon',
 'cattle_crush',
 'district',
 'highwater_mark',
 'locomotive',
 'ogham_stone',
 'railway_car',
 'road',
 'shieling',
 'ship',
 'tank',
 'vehicle',
 'wayside_cross',
 'wayside_shrine',
 'wreck']

Inoltre escluderò anche alcune colonne dal dataset primario in modo da restringere ulteriormente il campo.

In [7]:
bacoli_data = bacoli_data[['historic', 'name', 'historic:civilization', 'geometry']]
bacoli_data = bacoli_data[~bacoli_data['historic'].isin(excluded_keys)]
bacoli_data = bacoli_data[bacoli_data['name'].notna()]
bacoli_data.reset_index(inplace=True)

bacoli_data

,element_type,osmid,historic,name,historic:civilization,geometry
0,node,1045290525,archaeological_site,Piscina Mirabilis,NaN,POINT (14.08028 40.79519)
1,node,1104926219,archaeological_site,Grotte dell'Acqua - Lago Fusaro,NaN,POINT (14.05955 40.82506)
2,node,1139094827,archaeological_site,Grotta della Dragonara,NaN,POINT (14.08506 40.78477)
3,node,1139094838,archaeological_site,Sacello degli Augustali,NaN,POINT (14.08460 40.78747)
4,node,1706624371,archaeological_site,Grotta di Cocceio,ancient_roman_-_Augustus_Emperator_period,POINT (14.06211 40.84534)
5,node,2235447711,memorial,Bunker,NaN,POINT (14.06457 40.82321)
6,node,2235447893,memorial,Bunker,NaN,POINT (14.06555 40.82358)
7,node,8755033944,ruins,Tempo di Iside,NaN,POINT (14.05002 40.84415)
8,way,124092891,archaeological_site,Parco Archeologico Terme di Baia,ancient_roman,"POLYGON ((14.07084 40.81923, 14.07072 40.81929..."
9,way,124093354,archaeological_site,Anfiteatro di Cuma,ancient_roman,"POLYGON ((14.05636 40.84248, 14.05598 40.84255..."


Purtroppo a causa del fatto che la colonna *historic:civilization* non è corrattamente campita non riuscirò a sfruttare correttamente i dati a disposizione per rispondere alla mia domanda iniziale, ma meglio di niente. 

Come avrete sicuramente notato, il dataset ha geometrie miste. Di seguito lo normalizzerò per fare in modo da avere solo punti.

In [8]:
bacoli_polygons = bacoli_data[bacoli_data['element_type']=='way']

bacoli_polygons

,element_type,osmid,historic,name,historic:civilization,geometry
8,way,124092891,archaeological_site,Parco Archeologico Terme di Baia,ancient_roman,"POLYGON ((14.07084 40.81923, 14.07072 40.81929..."
9,way,124093354,archaeological_site,Anfiteatro di Cuma,ancient_roman,"POLYGON ((14.05636 40.84248, 14.05598 40.84255..."
10,way,144917231,castle,Castello Aragonese di Baia,NaN,"POLYGON ((14.08025 40.81003, 14.08046 40.81018..."
11,way,240100401,archaeological_site,Tomba di Agrippina,NaN,"POLYGON ((14.08145 40.80291, 14.08133 40.80294..."
12,way,323122081,building,Casina Vanvitelliana,NaN,"POLYGON ((14.05844 40.81979, 14.05847 40.81983..."
13,way,406222066,archaeological_site,Tempio di Diana,NaN,"POLYGON ((14.07065 40.81926, 14.07043 40.81923..."
14,way,429218964,archaeological_site,Columbari,NaN,"POLYGON ((14.06482 40.81952, 14.06543 40.81914..."
15,way,429324871,archaeological_site,Monumental Park of Baia,NaN,"POLYGON ((14.07125 40.81337, 14.07098 40.81320..."
16,way,429338160,archaeological_site,Tempio di Mercurio,NaN,"POLYGON ((14.06978 40.81790, 14.06982 40.81792..."
17,way,429339087,archaeological_site,Tempio di Venere,NaN,"POLYGON ((14.07139 40.81624, 14.07152 40.81627..."


In [9]:
bacoli_polygons['centroid'] = bacoli_polygons.geometry.centroid
bacoli_polygons.drop(columns={'geometry'}, inplace=True)
bacoli_polygons.rename(columns={'centroid': 'geometry'}, inplace=True)

bacoli_polygons

/tmp/ipykernel_24721/311082464.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bacoli_polygons['centroid'] = bacoli_polygons.geometry.centroid
/home/max/.cache/pypoetry/virtualenvs/pygis_blog-iPs2SW5R-py3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/tmp/ipykernel_24721/311082464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

,element_type,osmid,historic,name,historic:civilization,geometry
8,way,124092891,archaeological_site,Parco Archeologico Terme di Baia,ancient_roman,POINT (14.07024 40.81724)
9,way,124093354,archaeological_site,Anfiteatro di Cuma,ancient_roman,POINT (14.05604 40.84199)
10,way,144917231,castle,Castello Aragonese di Baia,NaN,POINT (14.08032 40.81106)
11,way,240100401,archaeological_site,Tomba di Agrippina,NaN,POINT (14.08128 40.80285)
12,way,323122081,building,Casina Vanvitelliana,NaN,POINT (14.05842 40.81971)
13,way,406222066,archaeological_site,Tempio di Diana,NaN,POINT (14.07056 40.81914)
14,way,429218964,archaeological_site,Columbari,NaN,POINT (14.06500 40.81928)
15,way,429324871,archaeological_site,Monumental Park of Baia,NaN,POINT (14.07141 40.81283)
16,way,429338160,archaeological_site,Tempio di Mercurio,NaN,POINT (14.06987 40.81782)
17,way,429339087,archaeological_site,Tempio di Venere,NaN,POINT (14.07152 40.81612)


In [10]:
bacoli_final = pd.concat([bacoli_polygons, bacoli_data[bacoli_data['element_type']=='node']])
bacoli_final.drop(columns={'element_type'}, inplace=True)
bacoli_final.sort_values(by='name', inplace=True)
bacoli_final = gpd.GeoDataFrame(bacoli_final, geometry='geometry')

bacoli_final

,osmid,historic,name,historic:civilization,geometry
9,124093354,archaeological_site,Anfiteatro di Cuma,ancient_roman,POINT (14.05604 40.84199)
5,2235447711,memorial,Bunker,NaN,POINT (14.06457 40.82321)
6,2235447893,memorial,Bunker,NaN,POINT (14.06555 40.82358)
12,323122081,building,Casina Vanvitelliana,NaN,POINT (14.05842 40.81971)
10,144917231,castle,Castello Aragonese di Baia,NaN,POINT (14.08032 40.81106)
14,429218964,archaeological_site,Columbari,NaN,POINT (14.06500 40.81928)
2,1139094827,archaeological_site,Grotta della Dragonara,NaN,POINT (14.08506 40.78477)
4,1706624371,archaeological_site,Grotta di Cocceio,ancient_roman_-_Augustus_Emperator_period,POINT (14.06211 40.84534)
1,1104926219,archaeological_site,Grotte dell'Acqua - Lago Fusaro,NaN,POINT (14.05955 40.82506)
15,429324871,archaeological_site,Monumental Park of Baia,NaN,POINT (14.07141 40.81283)


In [11]:
type(bacoli_final)

geopandas.geodataframe.GeoDataFrame

Ora che il dataset è di soli punti cerchiamo di rispondere a qualche domanda.

## 1.1. Come sono raggruppati i siti?

In [12]:
bacoli_group = bacoli_final.groupby(by='historic').count()
bacoli_group.drop(columns={'name', 'historic:civilization', 'geometry'}, inplace=True)
bacoli_group.rename(columns={'osmid': 'count'}, inplace=True)
bacoli_group.sort_values(by='count', ascending=False, inplace=True)

bacoli_group

,count
historic,
archaeological_site,14
memorial,2
building,1
castle,1
ruins,1


## 1.2. Dove sono localizzati?

http://scitools.org.uk/cartopy/docs/latest/
https://scitools.org.uk/cartopy/docs/v0.15/examples/tube_stations.html

In [ ]:
#fig, ax = plt.subplots(figsize=(10, 10))
b_minx, b_miny, b_maxx, b_maxy = bacoli_final.total_bounds

imagery = OSM()
ax = plt.axes(projection=imagery.crs)
ax.set_extent((b_minx, b_miny, b_maxx, b_maxy))
ax.add_image(imagery, 12)

# for bacoli_index, bacoli_row in bacoli_final.iterrows():
#     coordinates = bacoli_row.geometry.xy
#     ax.scatter(
#         *coordinates,
#         s=50,
#         marker="o",
#         facecolor='red',
#         edgecolor='black'
#     )
#     # plt.text(
#     #     x=coordinates[0][0],
#     #     y=coordinates[1][0],
#     #     s=bacoli_row.name,
#     #     fontdict=dict(color='black', size=10),
#     #     bbox=dict(facecolor='white', alpha=0.75)
#     # )
# 
# plt.show()

# Conclusione

Ora noi sappiamo che è la linea in posizione 0, della nuova lista riordinata, ad essere quella meno lunga. Quello che hai visto è uno dei metodi per arrivare all'obiettivo, ce ne saranno sicuramente altri.